In [1]:
import numpy as np
from numpy import load
from tqdm import tqdm
from sklearn.metrics import ndcg_score

MeanAveragePrecision@K

In [17]:
def user_average_precision(user_relevances, k=20):
  if user_relevances[:k].sum() == 0:
    return 0
  else:
    # Считаем значение Average Precision@k для одного пользовтаеля.
    average_precision_list = list()

    for k_items in range(1, (k + 1)):
      precision = user_relevances[:k_items].sum()
      precision /= len(user_relevances[:k_items])
      average_precision_list.append(precision)

    average_precision_array = np.array(average_precision_list)
    average_precision = average_precision_array * user_relevances[:k]
    average_precision = average_precision.sum()
    average_precision /= user_relevances[:k].sum()

    return average_precision

In [18]:
def get_map(relevances, k=20):
  mean_average_precision = 0
  n = len(relevances)

  for i in tqdm(range(n)):
    average_precision = user_average_precision(relevances[i], k)
    mean_average_precision += average_precision

  return mean_average_precision / n

MeanAverageRecall@K

In [19]:
def eval_single(recommended_items_ids, relevant_items_ids, k=20):
  recall_at_k = sum(
      [
          1
          for rec_item in recommended_items_ids[:k]
          if rec_item in relevant_items_ids
      ]
  ) / min(len(relevant_items_ids), k)

  return recall_at_k

In [20]:
def user_average_recall(user_relevances_recommended, recommended_items_ids, relevant_items_ids, k=20):
  if user_relevances_recommended[:k].sum() == 0:
    return 0
  else:
    average_recall_list = list()

    for k_items in range(1, (k + 1)):
      recall_at_k = eval_single(recommended_items_ids, relevant_items_ids, k_items)
      average_recall_list.append(recall_at_k)

    average_recall_array = np.array(average_recall_list)
    average_recall_array = average_recall_array * user_relevances_recommended[:k]
    average_recall = average_recall_array.sum()
    average_recall /= min(len(relevant_items_ids), k)

    return average_recall

In [21]:
def get_mar(relevances_recommended, recommended_items_ids, relevant_items_ids, k=20):
  mean_average_recall = 0
  n = len(relevances_recommended)

  for i in tqdm(range(n)):
    average_recall = user_average_recall(relevances_recommended[i], recommended_items_ids[i], relevant_items_ids[i], k)
    mean_average_recall += average_recall

  return mean_average_recall / n

nDCG@K

In [22]:
def get_ndcg(ranks_array, relevances_array, is_higher_better=False, k=20):
  n = len(ranks_array)
  ndcg = 0

  # Если на вход подаются скоры, а не ранги, то необходимо указать параметр is_higher_better = True.
  if is_higher_better == False:
    for i in tqdm(range(n)):
      rank = k - ranks_array[i]
      ndcg += ndcg_score([relevances_array[i]], [rank], k=20)
    return ndcg / n

  elif is_higher_better == True:
    for i in tqdm(range(n)):
      ndcg += ndcg_score([relevances_array[i]], [ranks_array[i]], k=20)
    return ndcg / n

3 в 1

In [23]:
def get_metrics(ranks, relevances, recommended_items_ids, relevant_items_ids, is_higher_better=False, k=20):
  metrics = dict()

  key = "MAP@" + str(k)
  metrics[key] = get_map(relevances, k)

  key = "MAR@" + str(k)
  metrics[key] = get_mar(relevances, recommended_items_ids, relevant_items_ids, k)

  key = "NDCG@" + str(k)
  metrics[key] = get_ndcg(ranks, relevances, is_higher_better, k)

  return metrics

Векторы хранятся в формате numpy.array. Они уже отсортированы по убыванию скоров (по preds_array), т.е. первым идёт товар с наибольшим скором, последним - с наименьшим.

In [3]:
preds_array = load("/content/drive/MyDrive/WB School/L4/preds_array_sample.npy", allow_pickle=True)
recommended_items_ids_array = load("/content/drive/MyDrive/WB School/L4/recommended_items_ids_array_sample.npy", allow_pickle=True)
relevances_array = load("/content/drive/MyDrive/WB School/L4/relevances_array_sample.npy", allow_pickle=True)
relevant_items_ids_array = load("/content/drive/MyDrive/WB School/L4/relevant_items_ids_array_sample.npy", allow_pickle=True)

In [10]:
def lists_to_arrays(ranks, relevances, recommended_items_ids, relevant_items_ids):
  ranks_array = list()
  relevances_array = list()
  recommended_items_ids_array = list()
  relevant_items_ids_array = list()

  for i in tqdm(range(len(ranks))):
    rank = np.array(ranks[i]).astype(float)
    ranks_array.append(rank)

    rel = np.array(relevances[i]).astype(int)
    relevances_array.append(rel)

    rec_items = np.array(recommended_items_ids[i]).astype(int)
    recommended_items_ids_array.append(rec_items)

    rel_items = np.array(relevant_items_ids[i]).astype(int)
    relevant_items_ids_array.append(rel_items)

  ranks_array = np.array(ranks_array)
  relevances_array = np.array(relevances_array)
  recommended_items_ids_array = np.array(recommended_items_ids_array)
  relevant_items_ids_array = np.array(relevant_items_ids_array)

  return ranks_array, relevances_array, recommended_items_ids_array, relevant_items_ids_array

In [11]:
preds_array, relevances_array, recommended_items_ids_array, relevant_items_ids_array = lists_to_arrays(
    preds_array,
    relevances_array,
    recommended_items_ids_array,
    relevant_items_ids_array
)

100%|██████████| 20/20 [00:00<00:00, 54330.36it/s]
<ipython-input-10-7caced98b2e2>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  relevant_items_ids_array = np.array(relevant_items_ids_array)


In [24]:
get_metrics(
    preds_array,
    relevances_array,
    recommended_items_ids_array,
    relevant_items_ids_array,
    is_higher_better=True,
    k=20
)

100%|██████████| 20/20 [00:00<00:00, 1547.09it/s]


{'MAP@20': 0.36836213680253926,
 'MAR@20': 0.02665192743764172,
 'NDCG@20': 0.5673287941612568}

id пользователей, которые попали в выборку:

In [ ]:
users_ids = [3, 25, 35, 68, 89, 90, 97, 130, 133, 135, 136, 139, 142, 143, 146, 148, 149, 155, 164, 171]